In [ ]:
# Load libraries
import os, glob, json, xarray, nibabel
from joblib import Parallel, delayed
from tqdm import tqdm
import pandas as pd
import numpy as np

from nilearn.input_data import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure
from nilearn.plotting import plot_roi
from nilearn.image import mean_img

In [ ]:
# Set main directories.
aomic_dir = '/mnt/mbServerData/data/AOMIC/'
main_dir = '/raid/projects/Emin/AOMIC/'
output_dir = os.path.join(main_dir, 'preprocessing')

# Define aomic_datasets
aomic_datasets = ['ID1000']

# Define tasks
tasks = ['moviewatching']

In [ ]:
# Set some parameters
fwhm = 6
high_pass = 1/128
n_cores = 30
save_figs = True

# Atlas to parcellate mri images.
atlas_dir = os.path.join(main_dir, 'atlasses', 'shen_1mm_268_parcellation.nii')

In [ ]:
# Define important functions
def organize_confound_mat(confounds):
    """Constructs a confound mat comprising tCompCor compounds and
    24 motion parameters."""
    t_comp_cors = [c for c in confounds.columns if 't_comp_cor' in c]
    t_comp_cors = confounds.loc[:, t_comp_cors[0]:t_comp_cors[-1]]
    mot_params = confounds.loc[:, 'trans_x':'rot_z_power2']
    return pd.concat([mot_params, t_comp_cors], axis=1).fillna(0)

def extract_time_series(task, subdir, aomic_dir, aomic_dataset, atlas_dir, 
    tr=None, save_figs=True, output_figure_dir=None, verbose=True):
    """Extracts time series from brain images using a given parcellation atlas.
    While extracting the time series, it also performs confound correction 
    (e.g., removing tCompCors and 24 motion parameters), high-pass filtering and smoothing."""
    sub_name = subdir[-8:]

    if verbose:
        print(f'Subject: {sub_name}\r', end='', flush=True)

    task_data_list = glob.glob(os.path.join(subdir, 'func', f'{sub_name}_task-{task}*'))

    if task_data_list:
        # task_sub_list.append(sub_name)
        for task_data_dir in task_data_list:
            if 'MNI152NLin2009cAsym_desc-preproc_bold.nii.gz' in task_data_dir:
                # find fmri image dir.
                mri_dir = task_data_dir
            if 'confounds_regressors.tsv' in task_data_dir:
                # Find confound dir.
                confound_dir = task_data_dir

        # Extract TR from imaging json file.
        if tr is None:
            img_info_dir = glob.glob(
                os.path.join(
                    aomic_dir, aomic_dataset, sub_name, 'func', f'{sub_name}_task-{task}*_bold.json'))[0]
            with open(img_info_dir) as f:
                tr = json.load(f)['RepetitionTime']

        # Save figure showing MRI images overlayed with atlas if desired.
        if save_figs:
            assert output_figure_dir is not None, 'You need to enter output directory for figures!'
            img = nibabel.load(mri_dir)
            plot_roi(
                atlas_dir, mean_img(img), title=sub_name, 
                output_file=os.path.join(output_figure_dir, f'{sub_name}.png'))

        # Prepare confound mat and extract time series.
        confound_mat = organize_confound_mat(pd.read_csv(confound_dir, delimiter='\t'))
        time_series = NiftiLabelsMasker(
            atlas_dir, high_pass=high_pass, t_r=tr, smoothing_fwhm=fwhm, standardize=True).fit_transform(mri_dir, confound_mat)
        # task_time_series.append(time_series)
        return sub_name, time_series

def construct_connectivity_matrices(time_series, corr_type='correlation', vectorize=False):
    """Constructs connectivity matrices using given time series."""
    conn = ConnectivityMeasure(kind = corr_type, vectorize=vectorize, discard_diagonal=True)
    return conn.fit_transform(np.array(time_series))


In [ ]:
# Extract time series from the MRI images using given atlas.
# Smoothing, high-pass filtering and confound regression are also applied.
tr = 2.2
for aomic_dataset in aomic_datasets:
    print(f'Dataset: {aomic_dataset}')
    sub_list = [f for f in glob.glob(os.path.join(aomic_dir, aomic_dataset,'derivatives','fmriprep','sub-*')) if os.path.isdir(f)]

    task_data_dict = {}
    for task in tasks:    
        print(f'\nTask: {task}')
        task_time_series = []
        task_sub_list = []      

        if save_figs: 
            output_figure_dir = os.path.join(output_dir, aomic_dataset, 'figures', task,'')
            if not os.path.exists(output_figure_dir):
                os.makedirs(output_figure_dir)
            
        results = Parallel(n_jobs=n_cores)(delayed(extract_time_series)(
            task, subdir, aomic_dir, aomic_dataset, atlas_dir, tr, True, output_figure_dir) for subdir in tqdm(sub_list))
        results = list(zip(*list(filter(None, results))))
        task_data_dict[task] = xarray.Dataset(
            {'time_series': xarray.DataArray(list(results[1]), dims=('ID', 'timeseries', 'nodes')), 'ID': list(results[0])})

    # Save time series into a netCDF file.
    for t in task_data_dict:
        task_data_dict[t].to_netcdf(os.path.join(output_dir, aomic_dataset, f'time_series_{t}.nc'))

print('\nDone')


In [ ]:
# Construct connectivity matrices using the timeseries.
# Pearson-correlation coefficient.
for aomic_dataset in aomic_datasets:
    print(f'Dataset: {aomic_dataset}')
    for task in tasks: 
        print(f'Task: {task}')
        # Define directory for time series.
        time_series_dir = os.path.join(output_dir, aomic_dataset, f'time_series_{task}.nc')
        time_series = xarray.open_dataset(time_series_dir) # Load time series data.

        # Compute vectorized connectivity matrices and save it into a .csv file.
        conn_mat_vectorized = construct_connectivity_matrices(time_series.time_series, vectorize=True)
        pd.concat(
            [pd.Series(time_series.ID), pd.DataFrame(conn_mat_vectorized)], axis=1).to_csv(
                os.path.join(output_dir, aomic_dataset, f'conn_mat_vectorized_{task}.csv'), index=None
            )

        # Compute connectivity matrices (symmetric matrix) and save it into a .nc file.
        time_series['conn_mat'] = (('ID', 'nodes', 'nodes'), 
            construct_connectivity_matrices(time_series.time_series))
        time_series.drop('time_series').to_netcdf(
            os.path.join(output_dir, aomic_dataset, f'conn_mat_{task}.nc'))
        
print('Done!!!')       
    